COGS 108 - Final Project

Overview

My project analyzes the relationship between the ratings San Diego Parks receive on Yelp and the amount of crime that is reported nearby the park. My main aim was to test whether a negative relationship between increasing crime and decreasing Yelp reviews, as crime infested areas tend to be less taken care of and a lot more dangerous.

Name & GitHub
Name: Jayush Arora
GitHub Username: jayusharora

Research Question
Is there any correlation between San Diego crime and San Diego negative Yelp reviews?

Background and Prior Work
San Diego Parks and Recreation has been in service since 1913, providing San Diego citizens a public place to relax and enjoy the nice weather offered by southern California. The city spends millions of dollars maintaining these parks, keeping them clean and safe for everyone. In fact, a new children's park can take upwards of 8 million dollars of an investment. Crime in San Diego, however, has also gone up over the last few decades, and shows no sign of decline.

Yelp is a very common place for people to go to leave reviews, and is the first review search Google shows you on their pages. Hence, Yelp reviews have a large weight on the success or failure of a particular public attraction, such as a park.

My study aims to show whether a relationship occurs between crime rates near parks in San Diego and the negative Yelp reviews left on these parks.
References (include links):
1) https://www.sandiego.gov/park-and-recreation/centers/map
2) https://www.sandiego.gov/police/services/statistics
Hypothesis
I hypothesize that a linear model can explain the correlation between San Diego crime and San Diego negative park reviews on Yelp.
Dataset(s)
As I was provided with all the data used in this project, I cannot account for how reliable it is.
Dataset Name: yelp_SD_Parks.csv
Link to the dataset: https://github.com/jayusharora/individual_fa20/blob/master/data/yelp_SD_parks.csv
Number of observations: 833
Description: This dataset includes information about all San Diego Parks and related centers, including name, address, rating, number of reviews, etc
Dataset Name: pd_calls_for_service_2019_datasd.csv
Link to the dataset: https://github.com/jayusharora/individual_fa20/blob/master/data/pd_calls_for_service_2019_datasd.csv
Number of observations: 590967
Description: This dataset includes information about every police call made in San Diego in 2019, including information like location, time, type of call, priority, etc

In order to combine these datasets, I had to find out which police calls were located near a San Diego park.


In [1]:
import numpy as np #for arithmetic operations
import pandas as pd #for table manipulations
from scipy import stats #for statistical analysis
import seaborn as sns #for data visualization
import matplotlib.pyplot as plt #for data visualization
from sklearn import linear_model
from sklearn.metrics import r2_score
# is there any correlation between san diego crime and san diego park negative yelp reviews for 2019 and 2020?

In [2]:
pd_calls_2019_filepath = "https://raw.githubusercontent.com/jayusharora/individual_fa20/master/data/pd_calls_for_service_2019_datasd.csv"
pd_calls_2019_df = pd.read_csv(pd_calls_2019_filepath)

yelp_reviews_filepath = "https://raw.githubusercontent.com/jayusharora/individual_fa20/master/data/yelp_SD_parks.csv"
yelp_reviews_df = pd.read_csv(yelp_reviews_filepath)

In [3]:
pd_calls_2019_df.head()

,incident_num,date_time,day_of_week,address_number_primary,address_dir_primary,address_road_primary,address_sfx_primary,address_dir_intersecting,address_road_intersecting,address_sfx_intersecting,call_type,disposition,beat,priority
0,E19010000001,2019-01-01 00:00:01,1,3800,NaN,NATIONAL,AVE,NaN,NaN,NaN,AU1,DUP,441,1
1,E19010000002,2019-01-01 00:00:04,1,6500,NaN,REFLECTION,DR,NaN,NaN,NaN,415,K,321,2
2,E19010000003,2019-01-01 00:00:09,1,0,NaN,06TH,AVE,NaN,ISLAND,NaN,FD,K,523,2
3,E19010000004,2019-01-01 00:00:19,1,0,NaN,VESTA,ST,NaN,MAIN,NaN,AU1,W,511,1
4,E19010000005,2019-01-01 00:00:31,1,0,NaN,04TH,AVE,NaN,ISLAND,NaN,586,CAN,523,4


In [4]:
yelp_reviews_df.head()

,name,address,phone,id,url,rating,review_count,longitude,latitude,is_closed
0,Balboa Park,"1549 El Prado San Diego, CA 92101",1.619239e+10,9M_FW_-Ipx93I36w-_ykBg,https://www.yelp.com/biz/balboa-park-san-diego...,5.0,2105,-117.153150,32.734502,False
1,Civita Park,"7960 Civita Blvd San Diego, CA 92108",NaN,3AEHjqNrTmggA6G9VdhQfg,https://www.yelp.com/biz/civita-park-san-diego...,4.5,46,-117.147278,32.778315,False
2,Waterfront Park,"1600 Pacific Hwy San Diego, CA 92101",1.619233e+10,3unbJeYrn1RmInZGmjp80g,https://www.yelp.com/biz/waterfront-park-san-d...,4.5,242,-117.172479,32.721952,False
3,Trolley Barn Park,"Adams Ave And Florida St San Diego, CA 92116",NaN,PvHxIYrmaiFKdWUDTMDzcg,https://www.yelp.com/biz/trolley-barn-park-san...,4.5,102,-117.143789,32.762463,False
4,Bay View Park,"413 1st St Coronado, CA 92118",NaN,6IF4VB9-fkv_F-LBvG8ppQ,https://www.yelp.com/biz/bay-view-park-coronad...,5.0,42,-117.178967,32.701785,False


In [5]:
yelp_reviews_df.columns
#interested in: name, address, rating, review_count, is_closed?
#conditions:
#name: parks specifically
#address: see if it matches any address in the pd databases

Index(['name', 'address', 'phone', 'id', 'url', 'rating', 'review_count',
       'longitude', 'latitude', 'is_closed'],
      dtype='object')

In [6]:
pd_calls_2019_df.columns
pd_calls_2020_df.columns
#interested in: 'address_road_primary', 'address_sfx_primary'

NameError: name 'pd_calls_2020_df' is not defined

In [ ]:
pd_calls_2019_df.drop(columns = ['incident_num', 'date_time', 'day_of_week', 'address_number_primary',
       'address_dir_primary', 'address_dir_intersecting', 'address_road_intersecting',
       'address_sfx_intersecting', 'call_type', 'disposition', 'beat',
       'priority'], inplace = True)

In [ ]:
yelp_reviews_df.dropna()
pd_calls_2019_df = pd_calls_2019_df.dropna()

In [ ]:
pd_calls_2019_df["address_road_primary"] = pd_calls_2019_df["address_road_primary"].str.lower()
pd_calls_2019_df["address_sfx_primary"] = pd_calls_2019_df["address_sfx_primary"].str.lower()

In [ ]:
pd_calls_2019_df.head()

In [ ]:
#function to get rid of all yelp reviews that are not about parks

parks_df = yelp_reviews_df[yelp_reviews_df["name"].str.contains("Park")]
parks_df.drop(columns = ["phone", "id", "url", "longitude", "latitude", "is_closed"], inplace = True)

In [ ]:
parks_df.head()

In [ ]:
parks_df["police_incidents"] = 0
parks_df = parks_df.astype({"name": str, "address" : str, "rating": float, "review_count": int})

In [ ]:
pd2019_df = pd_calls_2019_df["address_road_primary"] + " " + pd_calls_2019_df["address_sfx_primary"]
pd2019_df = pd.DataFrame(pd2019_df, columns = ["address"])

In [ ]:
parks_df["address"] = parks_df["address"].str.lower()

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd2019_df = pd2019_df.astype(str)
parks_df.head()

In [ ]:
for i in pd2019_df["address"]:
    street_count = parks_df["address"].str.contains(i)
    if len(parks_df[street_count].index) < 1:
        pass
    else:
        parks_df["police_incidents"][parks_df[street_count].index[0]] = sum(street_count == True)
    print(i, sum(street_count == True))

In [ ]:
parks_df

In [ ]:
# model the data
regr = linear_model.LinearRegression()
x = np.asanyarray(parks_df[["police_incidents"]])
y = np.asanyarray(parks_df[["rating"]])
regr.fit(x, y)
# the coefficients
print("Coefficients: ", regr.coef_)
print("Intercept: ", regr.intercept_)

In [ ]:
y_hat = regr.predict(x)

print("Variance score: %.95f" % regr.score(x, y))
print("Mean absolute error: %.95f" % np.mean(np.absolute(y_hat - y)))
print("Residual sum of squares (MSE): %.95f" % np.mean((y_hat - y) ** 2))
print("R2-score: %.95f" % r2_score(y_hat, y))

In [ ]:
plt.scatter(x, y)

In [ ]:
sns.lineplot(data=parks_df, x="police_incidents", y="rating")

In [ ]:
sns.pairplot(parks_df)

Ethics & Privacy
As this project did not require any personal information about any people, there was no use of any personal or harmful information, which keeps this study ethical and private. This information could be used unethically to figure out which areas of San Diego receive less police activity, but that is unrelated to the study topic.

Conclusion & Discussion
To restate my question, I analyzed (using a linear model) whether there is a correlation between the crime in San Diego and the negative Yelp reviews on San Diego Parks. Given our R^2 value, my results are not statistically significant. A linear model would not be the best to model this relationship. More features are required for a different statistical model such as multi-linear regression to provide further evidence of a relationship between these two variables.
